<a href="https://colab.research.google.com/github/LAURYMEOW/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-20 01:17:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.35MB/s    in 0.2s    

2022-07-20 01:17:06 (6.35 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-------------------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...|2015-08-31 00:00:00|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun.

In [121]:
# Create the vine_table. DataFrame
from pyspark.sql.types import IntegerType

vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [122]:
vine_df=vine_df.withColumn("star_rating",vine_df["star_rating"].cast(IntegerType()))
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [123]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3I2DHQBR577SS|          5|            0|          0|   N|                Y|
|R1QNE9NQFJC2Y4|          5|            0|          0|   N|                Y|
|R3LIDG2Q4LJBAO|          5|            0|          0|   N|                Y|
|R3KSZHPAEVPEAL|          5|            0|          0|   N|                Y|
| RAI2OIG50KZ43|          5|            0|          0|   N|                Y|
|R1R30FA4RB5P54|          4|            0|          0|   N|                Y|
|R30IJKCGJBGPJH|          5|            0|          0|   N|                Y|
|R18GLJJPVQ1OVH|          5|            0|          0|   N|                Y|
| R8TVYIJXLYJT0|          5|            0|          0|   N|                Y|
|R1CJGF6M3PVHEZ|          1|            0|          2|   N|     

1.Create a new Data Frame to retrieve all the rows where the total_votes count is equal to or greater than 20.


In [124]:
vine_df=vine_df.filter("total_votes>=20")
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
| RZN43XRZ89IIJ|          3|           10|         27|   N|                N|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|     

2. New Data Frame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal or greater than 50%

In [125]:
# Import functions
import pyspark.sql.functions as f
from pyspark.sql.functions import round, col

vine_df=vine_df.withColumn("divide", (f.lit(f.col("helpful_votes")/f.col("total_votes")*100))).select("*",round("divide",2)).filter("divide>=50")
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           divide|round(divide, 2)|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|78.26086956521739|           78.26|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|83.33333333333334|           83.33|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|95.83333333333334|           95.83|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|             90.0|            90.0|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|97.36842105263158|           97.37|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y| 97.1830985915493| 

3. Filter the Data Frame and create a new Data Frame that retrieves all the rows where a review was written as part of the Vine Program (paid).

In [126]:
vine_yes=vine_df.filter("vine=='Y'")
vine_yes.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           divide|round(divide, 2)|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+----------------+
|R106V6GUNWRSSS|          5|          857|        889|   Y|                N|96.40044994375702|            96.4|
|R37N8KZS48W36A|          5|           36|         37|   Y|                N| 97.2972972972973|            97.3|
|R1LNEQ2MQE03PD|          5|           49|         66|   Y|                N|74.24242424242425|           74.24|
|R1ECPWPUNK36ES|          5|           29|         31|   Y|                N|93.54838709677419|           93.55|
|R16BUM7UQZLOM6|          3|          107|        112|   Y|                N|95.53571428571429|           95.54|
| R7NWIOCA5RVPR|          5|           19|         26|   Y|                N|73.07692307692307| 

4. Filter the Data Frame and create a new Data Frame that retrieves all the rows where a review was written was not part of the Vine Program (unpaid).

In [127]:
vine_no=vine_df.filter("vine=='N'")
vine_no.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           divide|round(divide, 2)|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|78.26086956521739|           78.26|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|83.33333333333334|           83.33|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|95.83333333333334|           95.83|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|             90.0|            90.0|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|97.36842105263158|           97.37|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y| 97.1830985915493| 

5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [138]:
#Total number of reviews where the helpful_votes/totalvotes is equal or greaer than 50%

vine_df.count()


74760

In [139]:
# Total number of reviews where there is a Vine Review (Paid)
vine_yes.count()

647

In [129]:
# Total number of reviews where there is a Vine Review (Paid) and there is 5-star rating.
vine_yes.filter("star_rating==5").count()

229

In [151]:
# Percentage between number of reviews where there is a Vine Review /Total number of reviews where the helpful_votes/totalvotes is equal or greaer than 50%.
(vine_yes.count()/vine_df.count())*100

0.8654360620652755

In [149]:
# Percentage between number of reviews where there is a Vine Review with 5-star rating/Total number of reviews paid

vine_yes.filter("star_rating==5").count()/vine_yes.count()*100

35.394126738794434

In [140]:
# Total number of reviews where there isn't a Vine Review (Unpaid)

vine_no.count()

74113

In [141]:
# Total number of reviews where there isn't a Vine Review (Unpaid) and there is 5-star rating

vine_no.filter("star_rating==5").count()

43217

In [150]:
# Percentage between number of reviews where there isn't a Vine Review with 5-star rating/Total number of reviews unpaid

vine_no.filter("star_rating==5").count()/vine_no.count()*100

58.312306882733125